# Network Prepare REVISED 4/08/24
In this step, we add signals, bicycle facilities (with dates), and elevation attributes (avg up/down slop), remove links where cyclists are not allowed, and create a psuedo graph edge list for modeling turns.

Further refinement of the attributes for impedance calibration pruposes should be done in the `impedance_calibration` module. This includes reconciling the different attributes.

In [47]:
import geopandas as gpd
from pathlib import Path
import numpy as np
import pandas as pd

import pickle
import src.modeling_turns as modeling_turns
import src.add_attributes as add_attributes
import src.prepare_network as prepare_network

import sys
sys.path.insert(0,str(Path.cwd().parent))
import file_structure_setup
config = file_structure_setup.filepaths()

## Import network links and add attributes back

In [48]:
links = gpd.read_file(config['network_fp'] /'networks.gpkg',layer='osm_links')
og_cols = links.columns
nodes = gpd.read_file(config['network_fp'] / 'networks.gpkg',layer='osm_nodes')

In [49]:
# #temporary
# #also give raceways this
# links.loc[links['highway']=='raceway','link_type'] = 'no_access_or_private'
# links.loc[links['highway'].isin(['construction','proposed']),'link_type'] = 'construction or proposed'
# links[og_cols].to_file(config['network_fp'] /'networks.gpkg',layer='osm_links')

In [50]:
#basic stats
print(links.shape[0],'links',(links.length.sum() / 5280).round(0),'miles',nodes.shape[0],'nodes')

129627 links 4268.0 miles 106786 nodes


In [51]:
#types and length
summary_df = pd.DataFrame({'size':links.groupby('link_type').size(),
                           'length_mi':links.groupby('link_type')['geometry'].apply(lambda x: x.length.sum() / 5280),
                           'length_pct':links.groupby('link_type')['geometry'].apply(lambda x: x.length.sum()) / links.length.sum() * 100})
summary_df.sort_values('length_mi',ascending=False)

,size,length_mi,length_pct
link_type,,,
road,50657,1957.762113,45.872864
service,33303,1001.116198,23.457430
no_access_or_private,7387,365.882754,8.573100
parking_and_driveways,17472,327.137301,7.665244
pedestrian,10279,260.741801,6.109513
restricted_access_road,841,190.783647,4.470304
sidewalk,9449,147.701124,3.460826
bike,238,16.663768,0.390453
construction or proposed,1,0.011368,0.000266


In [52]:
summary_df.sum()

size          129627.000000
length_mi       4267.800074
length_pct       100.000000
dtype: float64

## Import and add attributes

In [53]:
#add attributes back (especially the oneway column)
osm_attrs = gpd.read_file(config['osmdwnld_fp'] / f"osm_{config['geofabrik_year']}.gpkg",layer='raw')#ignore_geometry=True)

In [54]:
osm_attrs.shape[0]

60908

In [55]:
# get basic stats
osm_attrs.to_crs(links.crs,inplace=True)
(osm_attrs.length / 5280).sum()

4598.927912949249

In [56]:
links = pd.merge(links,osm_attrs.drop(columns=['geometry']),left_on='osmid',right_on='osmid')
del osm_attrs

## Turn oneway into boolean

In [57]:
links['oneway'] = links['oneway'].isin(['yes','-1'])

## Add bicycle infrastructure and dates (city of atlanta)

In [58]:
# #TODO temporary version while i get the dates added
# cycling_infra_dates = gpd.read_file(config['bicycle_facilities_fp']/'osm_cycleways_w_dates.gpkg',layer='test_dates',ignore_geometry=True)
# links = pd.merge(links,cycling_infra_dates[['osmid','facility_fwd','facility_rev','year']],on='osmid',how='left')
# del cycling_infra_dates

## Add network improvements (atlanta)

In [59]:
# improvements = gpd.read_file(config['bicycle_facilities_fp']/'network_improvements.gpkg',layer='coa',ignore_geometry=True)
# links = pd.merge(links,improvements,left_on='osm_linkid',right_on='linkid',how='left')
# links.drop(columns=['linkid'],inplace=True)

## Add bicycle infrastructure and dates (savannah)

In [60]:
improvements = gpd.read_file(config['bicycle_facilities_fp']/'network_improvements.gpkg',layer='savannah',ignore_geometry=True)
links = pd.merge(links,improvements,left_on='osm_linkid',right_on='linkid',how='left')
links

KeyError: 'linkid'

In [45]:
links

,osm_A,osm_B,osm_linkid,link_type,osmid,geometry,timestamp,version,type,highway,...,cycleway,service,footway,sidewalk,bicycle,foot,access,area,all_tags,geom_type
0,436633256,67075136,1125565548,restricted_access_road,37392645,"LINESTRING (893138.256 776938.164, 895183.009 ...",1663252114,17,way,motorway,...,NaN,NaN,NaN,NaN,no,NaN,NaN,NaN,"{""@changeset"": 0, ""@way_nodes"": [436633256, 68...",LineString
1,67075036,68560350,1125565549,restricted_access_road,9199408,"LINESTRING (904256.082 770151.801, 903129.103 ...",1663252114,22,way,motorway,...,NaN,NaN,NaN,NaN,no,NaN,NaN,NaN,"{""@changeset"": 0, ""@way_nodes"": [67075036, 685...",LineString
2,68527352,68527407,1125565550,road,9196359,"LINESTRING (901673.835 774900.749, 901716.150 ...",1671551474,8,way,unclassified,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"{""@changeset"": 0, ""@way_nodes"": [68527352, 685...",LineString
3,68527407,68527414,1125565552,road,9196359,"LINESTRING (904294.606 774340.621, 904372.853 ...",1671551474,8,way,unclassified,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"{""@changeset"": 0, ""@way_nodes"": [68527352, 685...",LineString
4,68527414,67023635,1125565562,road,9196359,"LINESTRING (907384.760 775511.496, 907542.686 ...",1671551474,8,way,unclassified,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"{""@changeset"": 0, ""@way_nodes"": [68527352, 685...",LineString
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
129622,10277959062,10277959063,1125695170,parking_and_driveways,1123883742,"LINESTRING (983143.028 755057.980, 983355.927 ...",1671629260,1,way,service,...,NaN,parking_aisle,NaN,NaN,NaN,NaN,NaN,NaN,"{""@changeset"": 0, ""@way_nodes"": [10277959062, ...",LineString
129623,10277959064,10277959065,1125695171,parking_and_driveways,1123883743,"LINESTRING (983197.892 755026.354, 983411.720 ...",1671629260,1,way,service,...,NaN,parking_aisle,NaN,NaN,NaN,NaN,NaN,NaN,"{""@changeset"": 0, ""@way_nodes"": [10277959064, ...",LineString
129624,10277959066,10277959067,1125695172,parking_and_driveways,1123883744,"LINESTRING (983269.269 754985.176, 983483.742 ...",1671629260,1,way,service,...,NaN,parking_aisle,NaN,NaN,NaN,NaN,NaN,NaN,"{""@changeset"": 0, ""@way_nodes"": [10277959066, ...",LineString
129625,10277959068,10277959069,1125695173,parking_and_driveways,1123883745,"LINESTRING (983326.064 754952.441, 983538.009 ...",1671629260,1,way,service,...,NaN,parking_aisle,NaN,NaN,NaN,NaN,NaN,NaN,"{""@changeset"": 0, ""@way_nodes"": [10277959068, ...",LineString


In [ ]:
links.drop(columns=['linkid'],inplace=True)

## Add GDOT data

In [ ]:
gdot_lanes = gpd.read_file(config['network_fp']/"conflation.gpkg",layer="gdot_lanes",ignore_geometry=True)
gdot_traffic = gpd.read_file(config['network_fp']/"conflation.gpkg",layer="gdot_traffic",ignore_geometry=True)

In [ ]:
links = pd.merge(links,gdot_lanes,on="osmid",how='left')
links = pd.merge(links,gdot_traffic,on='osmid',how='left')

In [ ]:
links.columns

## Add HERE data

In [ ]:
# here = gpd.read_file(config['network_fp']/"conflation.gpkg",layer="here",ignore_geometry=True)
# links = pd.merge(links,here,on='osm_linkid',how='left')

## Add Bike Ottawa LTS

In [ ]:
# lts_paths = (Path.home() / "Documents/GitHub/stressmodel/data").glob("*.json")
# lts_list = []
# for lts_path in lts_paths:
#     lts_edges = gpd.read_file(lts_path,ignore_geometry=True)
#     lts_edges['osmid'] = lts_edges['id'].str.split('/').apply(lambda x: x[1]).astype(int)
#     lts = int(lts_path.name.split('_')[1].split('.')[0])
#     lts_edges['lts'] = lts
#     lts_list.append(lts_edges[['osmid','lts']])
# lts_df = pd.concat(lts_list)
# links = pd.merge(links,lts_df,on='osmid',how='left')

## Add elevation data
Assign the correct direction, then flip signs for reverse links later

In [ ]:
elevation = gpd.read_file(config['network_fp']/'elevation.gpkg',layer='elevation',ignore_geometry=True)
ascent_columns = [col for col in elevation.columns if "ascent" in col]
descent_columns = [col for col in elevation.columns if "descent" in col]
links = pd.merge(links,elevation[['osm_linkid']+ascent_columns+descent_columns],on='osm_linkid',how='left')
del elevation

In [ ]:
#UPDATE: fixed the reverse links bug so no longer need to flip the geometry here

In [ ]:
# #first take the absolute value then flip the elevations to match the geometry
# #TODO do this in the elevation script instead
# links.loc[:,ascent_columns+descent_columns] = links.loc[:,ascent_columns+descent_columns].abs()
# links.loc[links['reverse_geometry']==True,ascent_columns+descent_columns] = links.loc[links['reverse_geometry']==True,descent_columns+ascent_columns].values

# #have to do this with bike facilities too
# #TODO, at some point figure this part out during network filtering
# #ONEWAY links do not have this issue
# links.loc[links['reverse_geometry']==True,['facility_fwd','facility_rev']] = links.loc[links['reverse_geometry']==True,['facility_rev','facility_fwd']].values

In [ ]:
#actually reverse the underlying geometry so it doesn't mess up the turns
#links.loc[links['reverse_geometry']==True,'geometry'] = links['geometry'].apply(lambda geom: geom.reverse())

In [ ]:
links.rename(columns={'osm_A':'A','osm_B':'B','osm_linkid':'linkid'},inplace=True)
nodes.rename(columns={'osm_N':'N'},inplace=True)

In [ ]:
#calculate link lengths
links['length_ft'] = links.length

## Remove non-routable links


In [ ]:
links['link_type'].unique()
keep_type = ['road', 'service', 'parking_and_driveways', 'pedestrian', 'bike', 'sidewalk_or_crossing']
links = links[links['link_type'].isin(keep_type)]

## Remove isolated links

In [ ]:
links, nodes = prepare_network.largest_comp_and_simplify(links,nodes)

# Create reverse links and turn dataframe

In [ ]:
#TODO change this to not create the turn graph (just make it an extra optional step)
## Create turn graph dataframe
directed_links, turns_df = modeling_turns.create_pseudo_dual_graph(links,'A','B','linkid','oneway')

## Add signals (do later)

In [ ]:
osm_signals = pd.read_parquet(config['network_fp']/'osm_signals.parquet')
osm_signals.columns

In [ ]:
osm_signals['signalized'].value_counts()

In [ ]:
turns_df = pd.merge(turns_df,osm_signals[['source_linkid','source_reverse_link','target_linkid','target_reverse_link','signalized']],on=['source_linkid','source_reverse_link','target_linkid','target_reverse_link'],how='left')

In [ ]:
turns_df.loc[turns_df['signalized'].isna(),'signalized'] = False

In [ ]:
turns_df['signalized'].value_counts()

In [ ]:
# finding stressful intersections
# import pandas as pd
# highway_order = {
#     'trunk': 0,
#     'trunk_link': 1,
#     'primary': 2,
#     'primary_link': 3,
#     'secondary': 4,
#     'secondary_link': 5,
#     'tertiary': 6,
#     'tertiary_link': 7,
#     'unclassified': 8,
#     'residential': 9
# }
# highway_order = pd.Series(highway_order)
# highway_order = highway_order.reset_index()
# highway_order.columns = ['highway','order']
# #add highway ranking based on the above
# pseudo_df['target_highway_order'] = pseudo_df['target_highway'].map(highway_order.set_index('highway')['order'])
# pseudo_df['source_highway_order'] = pseudo_df['source_highway'].map(highway_order.set_index('highway')['order'])
# #remove straight and uturn
# cond1 = pseudo_df['turn_type'].isin(['left','right'])
# #only road to road for now
# cond2 = (pseudo_df['source_link_type'] == 'road') & (pseudo_df['target_link_type'] == 'road')
# cross_streets = pseudo_df[cond1 & cond2]

# #use groupby to find the max target_highway order
# cross_streets = cross_streets.groupby(['source_linkid','source_A','source_B'])['target_highway_order'].min()
# cross_streets.name = 'cross_street'

# #add to main df
# pseudo_df = pd.merge(pseudo_df,cross_streets,left_on=['source_linkid','source_A','source_B'],right_index=True,how='left')

# #change numbers back to normal
# pseudo_df['cross_street_order'] = pseudo_df['cross_street']
# pseudo_df['cross_street'] = pseudo_df['cross_street'].map(highway_order.set_index('order')['highway'])

In [ ]:
# add directional attributes and flip as needed
directed_links = pd.merge(directed_links,links[['linkid','facility_fwd','facility_rev']+ascent_columns+descent_columns],on='linkid')
directed_links.loc[directed_links['reverse_link']==True,ascent_columns+descent_columns] = directed_links.loc[directed_links['reverse_link']==True,descent_columns+ascent_columns].values
directed_links.loc[directed_links['reverse_link']==True,['facility_fwd','facility_rev']] = directed_links.loc[directed_links['reverse_link']==True,['facility_rev','facility_fwd']].values

In [ ]:
turns_df.to_parquet(config['network_fp']/'turns_df.parquet')
directed_links.to_parquet(config['network_fp']/'directed_edges.parquet')
links.to_file(config['network_fp']/'final_network.gpkg',layer='edges')
nodes.to_file(config['network_fp']/'final_network.gpkg',layer='nodes')

In [ ]:
#optional add geo data to turns and export for examination
from shapely.ops import MultiLineString
geo_dict = dict(zip(links['linkid'],links['geometry']))
turns_df['source_geo'] = turns_df['source_linkid'].map(geo_dict)
turns_df['target_geo'] = turns_df['target_linkid'].map(geo_dict)
turns_df['geometry'] = turns_df.apply(lambda row: MultiLineString([row['source_geo'],row['target_geo']]),axis=1)
turns_df.drop(columns=['source_geo','target_geo'],inplace=True)
turns_gdf = gpd.GeoDataFrame(turns_df,crs=links.crs)
turns_gdf.drop(columns=['source','target']).to_file(config['network_fp']/'final_network.gpkg',layer='turns')

In [ ]:
# #TODO serialize the attributes to add as needed?
# with (config['network_fp'] / 'edges_with_attributes.pkl').open('wb') as fh:
#     pickle.dump(links,fh)

In [ ]:
# with (config['network_fp'] / 'edges.pkl').open('wb') as fh:
#     pickle.dump(links,fh,protocol=pickle.HIGHEST_PROTOCOL)
# with (config['network_fp'] / 'nodes.pkl').open('wb') as fh:
#     pickle.dump(nodes,fh,protocol=pickle.HIGHEST_PROTOCOL)
# with (config['network_fp'] / 'directed_edges.pkl').open('wb') as fh:
#     pickle.dump(edges,fh,protocol=pickle.HIGHEST_PROTOCOL)
# with (config['network_fp'] / 'turn_df.pkl').open('wb') as fh:
#     pickle.dump(turn_df,fh,protocol=pickle.HIGHEST_PROTOCOL)

# Deprecated past here

In [ ]:

# #add attributes back and then flip elevation/bicyccle attributes
# #do so i don't have to re-flip everytime i import? could potentially save memory though
# #TODO it would still be smarter to store as a dict or something
# edges
# links.columns
# edges = pd.merge(edges,links.drop(columns=['A','B']),on='linkid')
# #if reverse_geo == true then ascent should be descent and vice versa
# # loops have reverse_geometry is np.nana
# # assume that all elevation columns will be paired by what is after ascent/descent
# elevation_columns = ['ascent_m', 'descent_m', 'ascent_grade_%','descent_grade_%']
# # Remove elements containing "ascent" or "descent"
# cleaned_columns = [col for col in elevation_columns if "ascent" not in col and "descent" not in col]    
# # Remove duplicates by converting the list to a set and back to a list
# cleaned_columns = list(set(cleaned_columns))

# for cleaned_column in cleaned_columns:
#     #swap if reverse geometry == true
#     links.loc[links['reverse_geometry']==True,ascent_columns+descent_columns] = links.loc[links['reverse_geometry']==True,descent_columns+ascent_columns]
    
    
#     df_edges[] = np.where(df_edges['reverse_link'], df_edges[elev_columns[1]].abs(), df_edges[elev_columns[0]])
#     #drop the down version?
#     df_edges.drop(columns=elev_columns[1],inplace=True)
# #if reverse_geo == true then ascent should be descent and vice versa
# # loops have reverse_geometry is np.nan
# # assume that all elevation columns will be paired by what is after ascent/descent
# elevation_columns = ['ascent_m', 'descent_m', 'ascent_grade_%','descent_grade_%']
# # Remove elements containing "ascent" or "descent"
# cleaned_columns = [col for col in elevation_columns if "ascent" not in col and "descent" not in col]    
# # Remove duplicates by converting the list to a set and back to a list
# cleaned_columns = list(set(cleaned_columns))

# for cleaned_column in cleaned_columns:
#     #swap
    
    
    
#     df_edges[] = np.where(df_edges['reverse_link'], df_edges[elev_columns[1]].abs(), df_edges[elev_columns[0]])
#     #drop the down version?
#     df_edges.drop(columns=elev_columns[1],inplace=True)
# ## Rename columns

# links.rename(columns={'osm_A':'A','osm_B':'B','osm_linkid':'linkid'},inplace=True)
# nodes.rename(columns={'osm_N':'N'},inplace=True)
# ## 
# ## Create turn graph dataframe
# edges, turn_df = modeling_turns.create_pseudo_dual_graph(links,'A','B','linkid','oneway')
# ## Flip attributes if needed (elevation, bicycle facilities)
# Turns should be good as is
# #add geo (needed for map matching part)
# df_edges = df_edges.merge(links.drop(columns=['A','B']),on=['linkid'])
# df_edges = gpd.GeoDataFrame(df_edges,geometry='geometry',crs=links.crs)
# df_edges = df_edges.loc[:,~df_edges.columns.duplicated()].copy()
# df_edges.reset_index(drop=True,inplace=True)
# #just export the df_edges?
# df_edges.to_file(export_fp/'Map_Matching/matching.gpkg',layer='edges')
# nodes.to_file(export_fp/'Map_Matching/matching.gpkg',layer='nodes')
# pseudo_df.columns
# #add geo to the turns too
# from shapely.ops import MultiLineString
# pseudo_df = pseudo_df.merge(links[['linkid','geometry']],left_on='source_linkid',right_on='linkid')
# pseudo_df = pseudo_df.merge(links[['linkid','geometry']],left_on='target_linkid',right_on='linkid')

# geometry = pseudo_df.apply(lambda row: MultiLineString([row['geometry_x'],row['geometry_y']]),axis=1)
# pseudo_df.drop(columns=['geometry_x','geometry_y','linkid_x','linkid_y'],inplace=True)
# pseudo_df = gpd.GeoDataFrame(pseudo_df,geometry=geometry,crs=links.crs)

# # pseudo_edges = pseudo_edges.loc[:,~pseudo_edges.columns.duplicated()].copy()
# # pseudo_edges.reset_index(drop=True,inplace=True)
# pseudo_df['source'] = pseudo_df['source'].astype(str)
# pseudo_df['target'] = pseudo_df['target'].astype(str)
# pseudo_df.to_file(export_fp/'Map_Matching/matching.gpkg',layer='turns')
# #pickle the graph
# with (export_fp / 'Map_Matching/turn_G.pkl').open('wb') as fh:
#     pickle.dump(pseudo_G,fh)
# # Come back to below later
# # Network Prepare
# This notebook prepares the final routing network.

# 1. Import the desired routing network
# 1. Add attributes
# 1. Add reconciled attributes
# 1. Add signals
# 1. Add elevation

# Then the network will be turned into a directed network graph complete with an edge list representing the directed edges and another one representing turns. Some attribute values are reversed to account for direction (e.g., elevation, signals).
# Import the data from previous notebooks and merge them. Merge here so updates can be done at each step without having to repeat everything.
# network_filepath = Path.home() / "Documents/BikewaySimData/Projects/gdot/networks"
# #filtered data
# links = gpd.read_file(network_filepath/'filtered.gpkg',layer='osm_links')
# nodes = gpd.read_file(network_filepath/'filtered.gpkg',layer='osm_nodes')
# links.columns
# #add osm data
# links = add_attributes.add_osm_attr(links,network_filepath / 'osm_attr.pkl')
# #rename
# links.rename(columns={'osm_A':'A','osm_B':'B','osm_linkid':'linkid'},inplace=True)
# nodes.rename(columns={'osm_N':'N'},inplace=True)
# links.columns
# #reconciled data
# reconciled = gpd.read_file(network_filepath/'reconciled.gpkg',layer='links',ignore_geometry=True)
# #[col for col in reconciled.columns if col not in links.columns]
# cols_to_keep = ['osm_linkid','speedlimit_range_mph','lanes_per_direction']
# links = links.merge(reconciled[cols_to_keep],on='osm_linkid',how='left')
# del reconciled
# #rename
# links.rename(columns={'osm_A':'A','osm_B':'B','osm_linkid':'linkid'},inplace=True)
# nodes.rename(columns={'osm_N':'N'},inplace=True)
# #signals added
# links_w_signals = gpd.read_file(network_filepath/'signals_added.gpkg',layer='links',ignore_geometry=True)

# nodes_w_signals = gpd.read_file(network_filepath/'signals_added.gpkg',layer='nodes',ignore_geometry=True)
# nodes_w_signals
# #TODO change linkid to osm_linkid later
# cols_to_keep = ['linkid','signal_A','signal_B']
# links = links.merge(links_w_signals[cols_to_keep],on='linkid',how='left')
# ##del nodes_w_signals

# #elevation added
# links_w_elevation = gpd.read_file(network_filepath/'elevation_added.gpkg',ignore_geometry=True)
# links_w_elevation.columns
# links_w_elevation.rename(columns={
#     'a_s_c_e_n_t___m':'ascent_m',
#     'd_e_s_c_e_n_t___m':'descent_m',
#     'a_s_c_e_n_t___g_r_a_d_e':'ascent_grade',
#     'd_e_s_c_e_n_t___g_r_a_d_e':'descent_grade',
# }, inplace =True)
# cols_to_keep = ['linkid','ascent_m','descent_m','ascent_grade','descent_grade','(0,2]_descent',
#        '(2,4]_descent', '(4,6]_descent', '(6,10]_descent', '(10,15]_descent',
#        '(15,inf]_descent', '(0,2]_ascent', '(2,4]_ascent', '(4,6]_ascent',
#        '(6,10]_ascent', '(10,15]_ascent', '(15,inf]_ascent']
# links = links.merge(links_w_elevation[cols_to_keep],on='linkid')
# del links_w_elevation
# links.columns
# fp = Path.home() / "Documents/BikewaySimData/Projects/gdot"
# edges = gpd.read_file(fp/'networks/elevation_added.gpkg',layer="links")


# edges.columns
# #use geometry one last time
# edges['length_ft'] = edges.length

# #turn bridge and tunnel to boolean values
# edges['tunnel'] = edges['tunnel'].notna()
# edges['bridge'] = edges['bridge'].notna()
# #turn bike facil into one column
# edges['bike_facility_type'] = np.nan
# edges.loc[(edges['mu'] == 1) & (edges['bike_facility_type'].isna()),'bike_facility_type'] = 'shared-use path'
# edges.loc[(edges['pbl'] == 1) & (edges['bike_facility_type'].isna()),'bike_facility_type'] = 'protected bike lane'
# edges.loc[(edges['bl'] == 1) & (edges['bike_facility_type'].isna()),'bike_facility_type'] = 'bike lane'
# df_edges, pseudo_df, pseudo_G = modeling_turns.create_pseudo_dual_graph(edges,'A','B','linkid','oneway',True)
# ## Add desired attributes from links to df_edges
# #df_edges = df_edges.merge(edges[['linkid','geometry']])

# basic_cols = ['linkid', 'osmid', 'link_type', 'name', 'oneway','length_ft']

# #anything that's an instance or would be better as a count value (but not a turn)
# event_cols = ['bridge','tunnel']

# #anything that's for the duration of the entire link and has categories
# category_cols = ['link_type','highway','speedlimit_range_mph',
#                'lanes_per_direction','bike_facility_type']

# #reverse in tuple form (these need to be flipped if going the other direction)
# rev_columns = [('ascent_m','descent_m'),
#                ('ascent_grade','descent_grade'),
#                ('(0,2]_ascent','(0,2]_descent'),
#                ('(2,4]_ascent','(2,4]_descent'),
#                ('(4,6]_ascent','(4,6]_descent'),
#                ('(6,10]_ascent','(6,10]_descent'),
#                ('(10,15]_ascent','(10,15]_descent'),
#                ('(15,inf]_ascent','(15,inf]_descent')]

# from itertools import chain
# keep_cols = basic_cols + event_cols + category_cols + list(chain(*rev_columns))
# # attrs = ['linkid', 'osmid', 'link_type', 'name', 'highway',
# #        'bridge', 'tunnel', 'bl', 'pbl', 'mu','speedlimit_range_mph',
# #        'lanes_per_direction', 'up_grade', 'down_grade', 'length_ft',
# #        'vehicle_separation','geometry']
# df_edges = df_edges.merge(edges[keep_cols],on='linkid',how='left')
# df_edges
# ## Deal with grade
# Need to flip sign of grade for reverse links
# # def combine_up_down_tuples(lst):
# #     result = []
# #     current_tuple = []

# #     for item in lst:
# #         if 'ascent' in item or 'descent' in item:
# #             current_tuple.append(item)
# #             if len(current_tuple) == 2:
# #                 result.append(tuple(current_tuple))
# #                 current_tuple = []

# #     return result

# # rev_columns = ['ascent_m','descent_m','ascent_grade','descent_grade',
# #                '(0,2]_down', '(2,4]_down', '(4,6]_down',
# #                '(6,10]_down', '(10,15]_down','(15,inf]_down',
# #                '(0,2]_up', '(2,4]_up', '(4,6]_up', '(6,10]_up',
# #                '(10,15]_up', '(15,inf]_up'
# #                ]

# # combined_tuples = combine_up_down_tuples(rev_columns)

# for elev_columns in rev_columns:
#     df_edges[elev_columns[0]] = np.where(df_edges['reverse_link'], df_edges[elev_columns[1]].abs(), df_edges[elev_columns[0]])
#     #drop the down version?
#     df_edges.drop(columns=elev_columns[1],inplace=True)
# ## Turns and Signals
# #add additional attributes needed for processing
# source_links = edges[['linkid','osmid','link_type','name','highway']]
# target_links = edges[['linkid','osmid','link_type','name','highway']]
# source_links.columns = 'source_' + source_links.columns
# target_links.columns = 'target_' + target_links.columns
# pseudo_df = pseudo_df.merge(source_links,on='source_linkid',how='left')
# pseudo_df = pseudo_df.merge(target_links,on='target_linkid',how='left')
# ## Turn Restrictions
# Two types in OSM (represented as OSM relations):
# - No (blank) turns
# - Only this turn allowed

# For chosen we don't need to consider turn restrictions
# # turn_restrictions = pd.read_csv(fp.parent/'osm_turn_restrictions.csv')
# # pseudo_df = pseudo_df.merge(turn_restrictions,left_on=['source_osmid','target_osmid'],right_on=['from_way_id','to_way_id'],how='left')
# # road_cond = (pseudo_df['source_link_type'] == 'road') & (pseudo_df['target_link_type'] == 'road')
# # no_restr = pseudo_df['type'] == 'no'
# # only_restr = pseudo_df['type'] == 'only'

# # #add a remove column
# # pseudo_df['remove'] = False

# # #remove the no turns
# # pseudo_df.loc[road_cond & no_restr,'remove'] = True

# # #for only, find all instances road_cond + from source and set to True
# # sources = set(turn_restrictions.loc[turn_restrictions['type']=='only','from_way_id'].tolist())
# # pseudo_df.loc[road_cond & pseudo_df['source_osmid'].isin(sources) & pseudo_df['type'].isna(),'remove'] = True

# # #Remove these turns and drop the added columns
# # print((pseudo_df['remove']==True).sum(),'turns removed')
# # pseudo_df = pseudo_df[pseudo_df['remove']==False]
# # pseudo_df.drop(columns=['relation_id', 'restriction', 'from_way_id',
# #        'to_way_id', 'type', 'remove'],inplace=True)
# # Deal with signals
# Perform two merges and use the source/target reverse link columns to determine which signal ID to keep.
# - For the source link, use signal_B if reverse == False else signal_A
# - For the target link, use signal_A if reverse == False else signal_B
# source = pseudo_df[['source_linkid','source_reverse_link']].merge(edges,left_on='source_linkid',right_on='linkid',how='left')
# pseudo_df['source_signal'] = np.where(source['source_reverse_link'], source['signal_A'], source['signal_B'])

# target = pseudo_df[['target_linkid','target_reverse_link']].merge(edges,left_on='target_linkid',right_on='linkid',how='left')
# pseudo_df['target_signal'] = np.where(target['target_reverse_link']==False, target['signal_B'], target['signal_A'])
# ## Identifying signalized/unsignalized turns
# - Only look at roads for now
# - Filter to left/right turns per source linkid per direction
# - Take the highest road classification and assign it as the cross street road classification
# import pandas as pd
# highway_order = {
#     'trunk': 0,
#     'trunk_link': 1,
#     'primary': 2,
#     'primary_link': 3,
#     'secondary': 4,
#     'secondary_link': 5,
#     'tertiary': 6,
#     'tertiary_link': 7,
#     'unclassified': 8,
#     'residential': 9
# }
# highway_order = pd.Series(highway_order)
# highway_order = highway_order.reset_index()
# highway_order.columns = ['highway','order']
# #add highway ranking based on the above
# pseudo_df['target_highway_order'] = pseudo_df['target_highway'].map(highway_order.set_index('highway')['order'])
# pseudo_df['source_highway_order'] = pseudo_df['source_highway'].map(highway_order.set_index('highway')['order'])
# #remove straight and uturn
# cond1 = pseudo_df['turn_type'].isin(['left','right'])
# #only road to road for now
# cond2 = (pseudo_df['source_link_type'] == 'road') & (pseudo_df['target_link_type'] == 'road')
# cross_streets = pseudo_df[cond1 & cond2]

# #use groupby to find the max target_highway order
# cross_streets = cross_streets.groupby(['source_linkid','source_A','source_B'])['target_highway_order'].min()
# cross_streets.name = 'cross_street'

# #add to main df
# pseudo_df = pd.merge(pseudo_df,cross_streets,left_on=['source_linkid','source_A','source_B'],right_index=True,how='left')

# #change numbers back to normal
# pseudo_df['cross_street_order'] = pseudo_df['cross_street']
# pseudo_df['cross_street'] = pseudo_df['cross_street'].map(highway_order.set_index('order')['highway'])
# # TODO Add OSM crossing into this logic
#     - Signals
#         - Wait on this until we have the route attributes code done
#         - Add crossings in signalization
#         - Majority of crossings are nodes not ways
#         - Cycleway crossings typically dealt the same way
#         - If meeting nodes are both crossings and within the traffic signal buffer, they're signalized crossings
#             - Or if both connecting links are crossings/connect to the road etc
#         - Way attributes
#             - Footway = crossing
#             - Highway = footway
#         - Node attributes
#             - Crossing = * (traffic signals/marked/etc)
#             - Highway = crossing
#         - Link attributes
#             - Some links are labeled as crossings but this is not as consistent

# signalized = pseudo_df['source_signal'] == pseudo_df['target_signal']
# left_or_straight =  pseudo_df['turn_type'].isin(['left','straight'])
# both_road = (pseudo_df['source_link_type'] == 'road') & (pseudo_df['target_link_type'] == 'road')
# cross_street = pseudo_df['cross_street_order'] <= 5

# #signalized
# pseudo_df.loc[signalized & both_road,'signalized'] = True
# pseudo_df.loc[pseudo_df['signalized'].isna(),'signalized'] = False
# # pseudo_df.loc[signalized & left_or_straight & both_road,'signalized_left_straight'] = True
# # pseudo_df.loc[pseudo_df['signalized_left_straight'].isna(),'signalized_left_straight'] = False

# pseudo_df.loc[-signalized & both_road & cross_street,'unsignalized'] = True
# pseudo_df.loc[pseudo_df['unsignalized'].isna(),'unsignalized'] = False

# #clean up
# rem =  ['source_osmid', 'source_link_type', 'source_name',
#        'source_highway', 'target_osmid', 'target_link_type', 'target_name',
#        'target_highway', 'source_signal', 'target_signal',
#        'target_highway_order', 'source_highway_order', 'cross_street',
#        'cross_street_order']
# pseudo_df.drop(columns=rem,inplace=True)
# # Export for impedance calibration

# # df_edges = gpd.GeoDataFrame(df_edges,crs='epsg:2240')
# df_edges.columns
# with (fp.parent / 'chosen.pkl').open('wb') as fh:
#     export = (df_edges,pseudo_df,pseudo_G)
#     pickle.dump(export,fh)
# ## Add geometry to examine results in QGIS
# #add geo
# link_geo = dict(zip(links['linkid'],links['geometry']))
# pseudo_df['src_geo'] = pseudo_df['source_linkid'].map(link_geo)
# pseudo_df['trgt_geo'] = pseudo_df['target_linkid'].map(link_geo)
# pseudo_df['geometry'] = pseudo_df[['src_geo','trgt_geo']].apply(lambda row: MultiLineString([row['src_geo'],row['trgt_geo']]),axis=1)

# pseudo_df.drop(columns=['src_geo','trgt_geo'],inplace=True)
# pseudo_df = gpd.GeoDataFrame(pseudo_df,crs=links.crs)

# pseudo_df['source'] = pseudo_df['source'].astype(str)
# pseudo_df['target'] = pseudo_df['target'].astype(str)

# #check results (may need a smaller road network to test on)
# pseudo_df.to_file(Path.home()/'Downloads/testing.gpkg',layer='cross_streets')